In [1]:
import pickle
from bs4 import BeautifulSoup as bs
from pprint import pprint as myprint

In [2]:
file = open(r"../tmp/bigram.txt","r",encoding="utf-8")
raw_tuple_list = [(line[:-3],line[-2]) for line in file.readlines()]
#"\n"为[line[-1]]
myprint(raw_tuple_list[:20])
file.close()

[('信息、信息媒体与信息资源', '1'),
 ('信息概述', '2'),
 ('信息的定义从广义上理解，信息可以被认为是物质的一种属性，是物质存在方式和运动规律与特点的表现形式。从狭义上理解，信息是一种消息、信号、数据或资料，是用来消除不定性的东西。',
  '3'),
 ('信息的特征', '3'),
 ('真实性', '4'),
 ('知识性', '4'),
 ('指向性', '4'),
 ('共享性', '4'),
 ('无限性', '4'),
 ('时效性', '4'),
 ('依附性', '4'),
 ('信息的种类', '3'),
 ('按信息的产生和作用机制划分，有自然信息和社会信息两大类；', '4'),
 ('按信息是否被人感知来划分，有显性信息和隐性信息两大类；', '4'),
 ('按信息的运动状态来划分，有动态信息和静态信息两大类；', '4'),
 ('按信息的存在和接受情况划分，有直接信息和间接信息两大类。', '4'),
 ('信息的功能与作用', '3'),
 ('维系社会存在与发展', '4'),
 ('社会经济发展的动力', '4'),
 ('科学决策、科学研究和生产建设的参谋和依据', '4')]


从bigram.txt文件中读入内容

In [3]:
raw_html_string = "<html><head></head><body></body></html>"
soup = bs(raw_html_string,"html5lib")
root = soup.html.body

简单的html代码，用于生成BeautiSoup对象，以供未来对该html文档树的操作

In [4]:
s1,r1 = raw_tuple_list[0]
new_tag = soup.new_tag("h%s"%r1)
new_tag.string = s1
root.append(new_tag)
print(root.prettify())

<body>
 <h1>
  信息、信息媒体与信息资源
 </h1>
</body>


把第一个节点加入到html文档树中，BeautifulSoup对象的new_tag()、string方法按照预期运行

In [5]:
curser = root.contents[0]
print(curser,type(curser))
#curser 用于跟踪当前节点
#目前root只有一个子节点，因此这样的写法完全没有问题      

<h1>信息、信息媒体与信息资源</h1> <class 'bs4.element.Tag'>


In [6]:
for i in range(len(raw_tuple_list)-1):
# for i in range(81,110):
# for i in range(100):
    s1,r1 = raw_tuple_list[i]
    s2,r2 = raw_tuple_list[i+1]
    if r1 < r2:
        new_tag = soup.new_tag("h%s"%r2)
        new_tag.string = s2
        curser.append(new_tag)
        curser = curser.contents[-1]
    if r1 == r2:
        new_tag = soup.new_tag("h%s"%r2)
        new_tag.string = s2
        curser = curser.parent        
        curser.append(new_tag)
        curser = curser.contents[-1]
    if r1 > r2:
        new_tag = soup.new_tag("h%s"%r2)
        new_tag.string = s2
        gap = int(r1) - int(r2)
        while gap >= 0:
            #bug
            if curser.parent == None:
                break
            #bug
            curser = curser.parent
            gap -= 1
        curser.append(new_tag)
        curser = curser.contents[-1]
#         print(r1,r2,s1[:10],curser.name)
        
        
print(root.prettify()[:500])

<body>
 <h1>
  信息、信息媒体与信息资源
  <h2>
   信息概述
   <h3>
    信息的定义从广义上理解，信息可以被认为是物质的一种属性，是物质存在方式和运动规律与特点的表现形式。从狭义上理解，信息是一种消息、信号、数据或资料，是用来消除不定性的东西。
   </h3>
   <h3>
    信息的特征
    <h4>
     真实性
    </h4>
    <h4>
     知识性
    </h4>
    <h4>
     指向性
    </h4>
    <h4>
     共享性
    </h4>
    <h4>
     无限性
    </h4>
    <h4>
     时效性
    </h4>
    <h4>
     依附性
    </h4>
   </h3>
   <h3>
    信息的种类
    <h4>
     按信息的产生和作用机制划分，有自然信息和社会信息两大类；
    </h4>
    <h4>
     按信息是否被人感知来划分，有显性信息和隐性信息两大类；
    </h4>
    


上面的思路很清楚，选择相邻的两个元组，比较其大纲级别，以构造其html文档树
这里还有有bug的，但是目前工作良好，问题在用#bug括起来的部分

In [7]:
html_file = open(r"../tmp/html.txt","w",encoding="utf-8")
html_file.write(soup.prettify())
html_file.close()

写html到文件，这个文件暂时每有什么用处，因为从文件中再次恢复成bs4对象，不能保证和当前的bs4对象一样
因此采用对象序列化的思路，将其保存在二进制文件，如下

In [8]:
html_file_b = open(r"../tmp/html.dat","wb")
pickle.dump(soup,html_file_b)
html_file_b.close()